In [5]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import BertTokenizer, Trainer, TrainingArguments, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
from transformers import pipeline

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ========= 1. 加载数据集 =========
dataset = load_dataset("imdb")  # 以IMDb数据集为例，你可以根据需要加载其他数据集
train_data = dataset["train"]
test_data = dataset["test"]

# ========= 2. 初始化 tokenizer 和模型 =========
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# ========= 3. 数据处理 =========
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_data = train_data.map(preprocess_function, batched=True)
test_data = test_data.map(preprocess_function, batched=True)

train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# ========= 4. 定义评估指标 =========
def compute_metrics(p):
    logits, labels = p.predictions, p.label_ids
    preds = np.argmax(logits, axis=1)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": accuracy, "f1": f1}

# ========= 5. 设置训练参数 =========
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=1,
)

# ========= 6. 定义 Trainer =========
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics,
)

# ========= 7. 开始训练 =========
train_results = trainer.train()

# ========= 8. 模型保存 =========
trainer.save_model("./multilingual_sentiment_model")

# ========= 9. 可视化训练过程 =========
# 获取损失和准确率
train_loss = train_results.training_loss
eval_results = trainer.evaluate()

# 绘制图表
plt.figure(figsize=(12, 6))
plt.plot(train_loss, label="Training Loss")
plt.plot(eval_results["eval_loss"], label="Validation Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# 绘制准确率
plt.figure(figsize=(12, 6))
plt.plot(eval_results["eval_accuracy"], label="Validation Accuracy")
plt.title("Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# ========= 10. 使用模型进行推理 =========
# 加载模型和tokenizer
model = BertForSequenceClassification.from_pretrained("./multilingual_sentiment_model")
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# 使用训练好的模型进行推理
sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# 测试数据
texts = ["我今天心情很好！", "这件事让我很生气", "天气有点冷，我感觉很中性"]

# 预测情感
results = sentiment_analyzer(texts)

# 打印结果
for text, result in zip(texts, results):
    print(f"Text: {text}")
    print(f"Sentiment: {result['label']}, Score: {result['score']:.4f}")


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 25000/25000 [01:47<00:00, 233.22 examples/s]
e:\demo\EmotionApp\backend\EmotionAppBackend\Algorithm\HuggingFace\hf\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
#检查标签分布：
from collections import Counter
print("训练集标签分布:", Counter(train_data['labels'].numpy()))
print("验证集标签分布:", Counter(eval_data['labels'].numpy()))

In [ ]:
model.save_pretrained('./go_emotions_model_2')
tokenizer.save_pretrained('./go_emotion_model_2')

In [ ]:
#学习率过高可能导致模型无法收敛，过低可能导致训练过慢。